# Training the best models
For this specific task, we found the best parameters for the differnt models and make it into a separate notebook for training

In [ ]:
import torch
import torch.nn as nn

from transformers import AlbertTokenizer, AutoTokenizer, AutoModelForSequenceClassification
from transformers import AlbertForSequenceClassification, BertConfig, TrainingArguments, Trainer

# For logging experiments when training
from transformers.integrations import NeptuneCallback, WandbCallback, TensorBoardCallback

from datasets import load_dataset
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
kb_bert = 'KB/bert-base-swedish-cased'
kb_electra = 'KB/electra-base-swedish-cased-discriminator'
kb_albert = 'KB/albert-base-swedish-cased-alpha'
mbert = 'bert-base-multilingual-cased'
xlmr = 'xlm-roberta-base'

#tokenizer = AutoTokenizer.from_pretrained(mbert)
#model = AutoModelForSequenceClassification.from_pretrained(mbert)

#tokenizer = AlbertTokenizer.from_pretrained(kb_albert)
#model = AlbertForSequenceClassification.from_pretrained(kb_albert)

#tokenizer = AutoTokenizer.from_pretrained(kb_electra)
#model = AutoModelForSequenceClassification.from_pretrained(kb_electra)

tokenizer = AutoTokenizer.from_pretrained(xlmr)
model = AutoModelForSequenceClassification.from_pretrained(xlmr)

In [ ]:
train_dataset = load_dataset("csv", data_files='../data/train2.csv')
test_dataset = load_dataset("csv", data_files='../data/test2.csv')

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True,  max_length = 512, add_special_tokens = True)


train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "steps",
    eval_steps = 250,
    logging_steps = 250,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,                       
    #warmup_steps=1000,                     # KB Alberta only
    learning_rate=5e-5,                     
    #learning_rate=3e-5,                    # KB Alberta only
    weight_decay=0.01,
    do_eval=True,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset['train'],
    eval_dataset=test_dataset['train'],
    callbacks=[NeptuneCallback],             # This will ensure that experiments are logged
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
#trainer.save_model("../models/kb_albert.pt")
#trainer.save_model("../models/mBert.pt")
#trainer.save_model("../models/kb_electra.pt")
trainer.save_model("../models/xlmr.pt")


In [ ]:
num_parameters = sum(p.numel() for p in model.parameters())

In [ ]:
num_parameters